In [6]:
#setup

import io
import os
import re
import sys
import shutil
import string
import tensorflow as tf

from datetime import datetime
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Activation, Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [7]:
imdb_dir = '/Users/dawidkubicki/Documents/dataset/aclImdb'
train_dir = os.path.join(imdb_dir, 'train')
os.listdir(train_dir)

['urls_unsup.txt',
 '.DS_Store',
 'neg',
 'urls_pos.txt',
 'urls_neg.txt',
 'pos',
 'unsupBow.feat',
 'labeledBow.feat']

In [8]:
#remove unused folders

# remove_dir = os.path.join(train_dir, 'unsup')
# shutil.rmtree(remove_dir)

In [9]:
batch_size = 1024
seed = 123
train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    '/Users/dawidkubicki/Documents/dataset/aclImdb/train', batch_size=batch_size, validation_split=0.2, 
    subset='training', seed=seed)
val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    '/Users/dawidkubicki/Documents/dataset/aclImdb/train', batch_size=batch_size, validation_split=0.2, 
    subset='validation', seed=seed)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [10]:
for text_batch, label_batch in train_ds.take(1):
    for i in range(4):
        print(label_batch[i].numpy(), text_batch.numpy()[i])

0 b"Oh My God! Please, for the love of all that is holy, Do Not Watch This Movie! It it 82 minutes of my life I will never get back. Sure, I could have stopped watching half way through. But I thought it might get better. It Didn't. Anyone who actually enjoyed this movie is one seriously sick and twisted individual. No wonder us Australians/New Zealanders have a terrible reputation when it comes to making movies. Everything about this movie is horrible, from the acting to the editing. I don't even normally write reviews on here, but in this case I'll make an exception. I only wish someone had of warned me before I hired this catastrophe"
1 b'This movie is SOOOO funny!!! The acting is WONDERFUL, the Ramones are sexy, the jokes are subtle, and the plot is just what every high schooler dreams of doing to his/her school. I absolutely loved the soundtrack as well as the carefully placed cynicism. If you like monty python, You will love this film. This movie is a tad bit "grease"esk (without

In [11]:
embedding_layer = tf.keras.layers.Embedding(1000,5) 

In [12]:
result = embedding_layer(tf.constant([1,2,3]))
result.numpy()

array([[ 0.00572307, -0.04319457,  0.04039507,  0.01173634,  0.04062193],
       [ 0.00248224,  0.02030558,  0.01864493, -0.03389881,  0.03394209],
       [ 0.00363629,  0.00429878, -0.04259536, -0.03929476, -0.02087897]],
      dtype=float32)

In [13]:
# Create a custom standardization function to strip HTML break tags '<br />'.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')

# Vocabulary size and number of words in a sequence.
vocab_size = 10000
sequence_length = 100

# Use the text vectorization layer to normalize, split, and map strings to 
# integers. Note that the layer uses the custom standardization defined above. 
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

In [14]:
embedding_dim=16

model = Sequential([
  vectorize_layer,
  Embedding(vocab_size, embedding_dim, name="embedding"),
  GlobalAveragePooling1D(),
  Dense(16, activation='relu'),
  Dense(1)
])

In [15]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [16]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [17]:
model.fit(
    train_ds,
    validation_data=val_ds, 
    epochs=15,
    callbacks=[tensorboard_callback])

Epoch 1/15
20/20 [==============================] - 5s 186ms/step - loss: 0.6922 - accuracy: 0.4996 - val_loss: 0.6884 - val_accuracy: 0.4886
Epoch 2/15
20/20 [==============================] - 2s 97ms/step - loss: 0.6866 - accuracy: 0.4979 - val_loss: 0.6806 - val_accuracy: 0.4886
Epoch 3/15
20/20 [==============================] - 2s 97ms/step - loss: 0.6776 - accuracy: 0.4962 - val_loss: 0.6687 - val_accuracy: 0.4886
Epoch 4/15
20/20 [==============================] - 2s 100ms/step - loss: 0.6631 - accuracy: 0.5009 - val_loss: 0.6499 - val_accuracy: 0.4886
Epoch 5/15
20/20 [==============================] - 2s 104ms/step - loss: 0.6416 - accuracy: 0.4974 - val_loss: 0.6270 - val_accuracy: 0.5080
Epoch 6/15
20/20 [==============================] - 2s 99ms/step - loss: 0.6148 - accuracy: 0.5344 - val_loss: 0.6003 - val_accuracy: 0.5712
Epoch 7/15
20/20 [==============================] - 2s 100ms/step - loss: 0.5842 - accuracy: 0.6127 - val_loss: 0.5708 - val_accuracy: 0.6514
Epoch 8/1

In [18]:
%tensorboard --logdir logs


UsageError: Line magic function `%tensorboard` not found.


In [20]:
weights = model.get_layer('embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [22]:
out_v = io.open('/Users/dawidkubicki/Documents/dataset/aclImdb/vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('/Users/dawidkubicki/Documents/dataset/aclImdb/metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if  index == 0: continue # skip 0, it's padding.
  vec = weights[index] 
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()